In [1]:
# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents


def convert_examples_to_features(examples, labels, max_seq_len, tokenizer, pad_token_id_for_segment=0,
                                     pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        tokens = []
        labels_ids = []
        for one_word, label_token in zip(example, label):
            #하나의 단어에 대해서 서브워드로 토큰화
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            #서브워드 중 첫번째 서브워드만 개체명 레이블을 부여하고 그 외에는 -100으로 채운다.
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))
            #[CLS]와[SEP]를 후에 추가할 것을 고려하여 최대 길이를 초과하는 샘플의 경 우 max_seq_len-2의 길이로 변환.
            #ex)max_seq_len=64라면 길이가62보다 긴 샘플은 뒷 부분을 자르고 길이 62로 변환.
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]
        #[SEP]를 추가하는 코드
        #1.토큰화 결과의 맨 뒷 부분에[SEP]토큰 추가 #2. 레이블에도 맨 뒷 부분에 -100 추가.
        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]
        #[CLS]를 추가하는 코드
        #1. 토큰화 결과의 앞 부분에 [CLS] 토큰 추가
        #2. 레이블의 맨 앞 부분에도 -100 추가.
        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids
        #정수 인코딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        #어텐션 마스크 생성
        attention_mask = [1] * len(input_id)
        #정수 인코딩에 추가할 패딩 길이 연산
        padding_count = max_seq_len - len(input_id)
        #정수 인코딩,어텐션 마스크에 패딩 추가
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        #세그먼트 인코딩.
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        # 레이블 패딩.(단, 이 경우는 패딩 토큰의 ID가 -100)
        label = labels_ids + ([pad_token_id_for_label] * padding_count)
        assert len(input_id) == max_seq_len, "Error with input length {} vs {}". format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}". format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
#!pip install transformers
#!pip install seqeval

In [6]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

#from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf
import urllib.request




# 학습용 말뭉치 데이터를 불러옴
#corpus = read_file('ner_train.txt')
corpus = read_file('mtn_ner_train_fullversion.txt')
print(corpus[0])


# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])

    sentences.append(sentence)
    tags.append(bio_tag)


print("샘플 크기 : \n", len(sentences))
print("0번 째 샘플 단어 시퀀스 : \n", sentences[0])
print("0번 째 샘플 bio 태그 : \n", tags[0])
print("샘플 단어 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 단어 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))


# 학습 데이터와 테스트 데이터를 8:2의 비율로 분리
train_data_sentence, test_data_sentence, train_data_label, test_data_label = train_test_split(sentences, tags,
                                                    test_size=.2,
                                                    random_state=1234)



[('1', '가락지빵', 'NNG', 'B_FOOD'), ('2', '주문', 'NNP', 'O'), ('3', '하', 'VV', 'O'), ('4', '고', 'EC', 'O'), ('5', '싶', 'VX', 'O'), ('6', '어요', 'EC', 'O')]
샘플 크기 : 
 65512
0번 째 샘플 단어 시퀀스 : 
 ['가락지빵', '주문', '하', '고', '싶', '어요']
0번 째 샘플 bio 태그 : 
 ['B_FOOD', 'O', 'O', 'O', 'O', 'O']
샘플 단어 시퀀스 최대 길이 : 168
샘플 단어 시퀀스 평균 길이 : 8.431951398217121


In [7]:
print(len(train_data_sentence))
print(len(test_data_label))
print(len(train_data_sentence))
print(len(test_data_label))
print(train_data_sentence[2])
print(train_data_label[2])

52409
13103
52409
13103
['머스터드', '주문', '오케이', '?']
['B_FOOD', 'O', 'O', 'O']


In [8]:
#labels = ['O', 'B_DT','B_FOOD', 'B_LC', 'I', 'B_OG', 'B_PS', 'NNP', 'B_TI', 'PAD']
labels = ['O', 'B_DT','B_FOOD', 'B_LC', 'I', 'B_OG', 'B_PS', 'NNP', 'B_TI']

tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

print(tag_to_index)
print(index_to_tag)

tag_size = len(tag_to_index)


{'O': 0, 'B_DT': 1, 'B_FOOD': 2, 'B_LC': 3, 'I': 4, 'B_OG': 5, 'B_PS': 6, 'NNP': 7, 'B_TI': 8}
{0: 'O', 1: 'B_DT', 2: 'B_FOOD', 3: 'B_LC', 4: 'I', 5: 'B_OG', 6: 'B_PS', 7: 'NNP', 8: 'B_TI'}


* 전처리

In [9]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

* 기존 토큰을 서브 토큰화
* 서브토큰에 매칭없는 라벨인코딩은 -100으로 처리
* -100은 PAD 처리

In [10]:
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 13103/13103 [00:08<00:00, 1602.10it/s]


In [11]:
print(len(X_train[:][0])) # ids, attention mask, token_type
print(len(y_train))
print(len(X_test))
print(len(y_test))

52409
52409
3
13103


In [12]:
print(' 기 존 원 문 :', train_data_sentence[0])
print(' 기 존 레 이 블 :', train_data_label[0])
print('-' * 50)
print(' 토 큰 화 후 원 문 :', [tokenizer.decode([word]) for word in X_train[0][0]])
print(' 토 큰 화 후 레 이 블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx
in y_train[0]])
print('-' * 50)
print(' 정 수 인 코 딩 결 과 :', X_train[0][0])
print(' 정 수 인 코 딩 레 이 블 :', y_train[0])

 기 존 원 문 : ['떡국', '미리', '주문', '하', 'ㄹ', '수', '있', '나요', '?']
 기 존 레 이 블 : ['B_FOOD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
--------------------------------------------------
 토 큰 화 후 원 문 : ['[CLS]', '떡국', '미리', '주문', '하', 'ㄹ', '수', '있', '나요', '?', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

In [13]:
print(' 세 그 먼 트 인 코 딩 :', X_train[2][0])
print(' 어 텐 션 마 스 크 :', X_train[1][0])

 세 그 먼 트 인 코 딩 : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 어 텐 션 마 스 크 : [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


* 모델링

In [14]:
class TFBertForTokenClassification(tf.keras.Model):

    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels, kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,token_type_ids=token_type_ids)

        # 전 체 시 퀀 스 에 대 해 서 분 류 해 야 하 므 로 outputs[0] 임 에 주 의
        all_output = outputs[0]
        prediction = self.classifier(all_output)

        return prediction


def compute_loss(labels, logits):
    # 다 중 클 래 스 분 류 문 제 에 서 소 프 트 맥 스 함 수 미 사 용 시 from_logits=True 로 설 정 .
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    # -100 의 값 을 가 진 정 수 에 대 해 서 는 오 차 를 반 영 하 지 않 도 록 labels 를 수 정 .
    active_loss = tf.reshape(labels, (-1,)) != -100
    # activa_loss 로 부 터 reduced_logits 과 labels 를 각 각 얻 는 다 .
    reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])) , active_loss)
    labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
    return loss_fn(labels, reduced_logits)

In [16]:
# TPU 작 동 을 위 한 코 드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.
environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
#strategy = tf.distribute.experimental.TPUStrategy(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model.compile(optimizer=optimizer, loss=compute_loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'bert.embeddings.position_ids', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [17]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test


    def sequences_to_tags(self, label_ids, pred_ids):
        label_list = []
        pred_list = []

        for i in range(0, len(label_ids)):
            label_tag = []
            pred_tag = []
            # 레 이 블 의 값 이 -100 인 경 우 는 F1 score 계 산 시 에 도 제 외
            # ex) 레 이 블 디 코 딩 과 정
            # label_index : [1 -100 2 -100] ===> [1 2] ===> label_tag : [PER-B PER-I]
            for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
                if label_index != -100:
                    label_tag.append(index_to_tag[label_index])
                    pred_tag.append(index_to_tag[pred_index])
            label_list.append(label_tag)
            pred_list.append(pred_tag)
        return label_list, pred_list

    # 에 포 크 가 끝 날 때 마 다 실 행 되 는 함 수 }
    def on_epoch_end(self, epoch, logs={}):
        y_predicted = self.model.predict(self.X_test)
        y_predicted = np.argmax(y_predicted, axis = 2)
        label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)
        score = f1_score(label_list, pred_list, suffix=True)
        print(' - f1: {:04.2f}'.format(score * 100))
        print(classification_report(label_list, pred_list, suffix=True))

In [18]:
f1_score_report = F1score(X_test, y_test)

In [19]:
model.fit(X_train, y_train, epochs=3, batch_size=32, callbacks = [f1_score_report])

Epoch 1/3


   6/1638 [..............................] - ETA: 1:59 - loss: 1.1329

410/410 [==============================] - 22s 41ms/step


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: B_FOOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:1

 - f1: 97.79
              precision    recall  f1-score   support

         B_D       0.99      0.99      0.99      4588
       B_FOO       1.00      1.00      1.00     11685
         B_L       0.93      0.95      0.94      1099
         B_O       0.81      0.81      0.81       632
         B_P       0.95      0.93      0.94       494
         B_T       0.94      0.86      0.90        78
          NN       1.00      1.00      1.00       297
           _       0.82      0.72      0.77       818

   micro avg       0.98      0.98      0.98     19691
   macro avg       0.93      0.91      0.92     19691
weighted avg       0.98      0.98      0.98     19691

1638/1638 [==============================] - 244s 96ms/step - loss: 0.0229
Epoch 2/3
410/410 [==============================] - 9s 21ms/step
 - f1: 98.11
              precision    recall  f1-score   support

         B_D       0.99      0.99      0.99      4588
       B_FOO       1.00      1.00      1.00     11685
         B_L       

In [22]:
# model save
model.save_weights('ner_model(bert)_use_cpsdic_mtndic_mtndat.h5')

In [23]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,pad_token_id_for_segment=0,pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []
    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            # 하 나 의 단 어 에 대 해 서 서 브 워 드 로 토 큰 화
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            # 서 브 워 드 중 첫 번 째 서 브 워 드 를 제 외 하 고 그 뒤 의 서 브 워 드 들 은 -100 으 로 채 운 다 .
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens)- 1))
        # [CLS] 와 [SEP] 를 후 에 추 가 할 것 을 고 려 하 여 최 대 길 이 를 초 과 하 는 샘 플 의 경우 max_seq_len - 2 의 길 이 로 변 환 .
        # ex) max_seq_len = 64 라 면 길 이 가 62 보 다 긴 샘 플 은 뒷 부 분 을 자 르 고 길 이 62 로 변 환 .
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]
        # [SEP] 를 추 가 하 는 코 드
        # 1. 토 큰 화 결 과 의 맨 뒷 부 분 에 [SEP] 토 큰 추 가
        # 2. 레 이 블 에 도 맨 뒷 부 분 에 -100 추 가 .
        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]
        # [CLS] 를 추 가 하 는 코 드
        # 1. 토 큰 화 결 과 의 앞 부 분 에 [CLS] 토 큰 추 가
        # 2. 레 이 블 의 맨 앞 부 분 에 도 -100 추 가 .
        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask
        # 정 수 인 코 딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        # 어 텐 션 마 스 크 생 성
        attention_mask = [1] * len(input_id)
        # 정 수 인 코 딩 에 추 가 할 패 딩 길 이 연 산
        padding_count = max_seq_len - len(input_id)
        # 정 수 인 코 딩 , 어 텐 션 마 스 크 에 패 딩 추 가
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        # 세 그 먼 트 인 코 딩 .
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        # 레 이 블 패 딩 . (단 , 이 경 우 는 패 딩 토 큰 의 ID 가 -100)

        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [24]:
X_pred, label_masks = convert_examples_to_features_for_prediction(
test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 1199.88it/s]


In [26]:
print(' 기 존 원 문 :', test_data_sentence[0])
print('-' * 50)
print(' 토 큰 화 후 원 문 :', [tokenizer.decode([word]) for word in X_pred[0][0]])
print(' 레 이 블 마 스 크 :', ['[PAD]' if idx == -100 else '[FIRST]' for idx in label_masks[0]])

 기 존 원 문 : ['금주', '6시', '40분', '단디', '먹', '고', '싶', '어요']
--------------------------------------------------
 토 큰 화 후 원 문 : ['[CLS]', '금주', '6', '##시', '40', '##분', '단', '##디', '먹', '고', '싶', '어요', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [27]:
def ner_prediction(examples, max_seq_len, tokenizer):

    examples = [sent.split() for sent in examples]
    X_pred, label_masks = convert_examples_to_features_for_prediction(examples,
    max_seq_len=128, tokenizer=tokenizer)
    y_predicted = model.predict(X_pred)
    y_predicted = np.argmax(y_predicted, axis = 2)
    pred_list = []
    result_list = []

    for i in range(0, len(label_masks)):
        pred_tag = []
        # ex) 모 델 의 예 측 값 디 코 딩 과 정
        # 예 측 값 (y_predicted) 에 서 레 이 블 마 스 크 (label_masks) 의 값 이 -100 인 동 일 위 치 의        값 을 삭 제
        # label_masks : [-100 0 -100 0 -100]
        # y_predicted : [ 0 1 0 2 0 ] ==> [1 2] ==> 최 종 예 측 (pred_tag) : [PERB    PER-I]
        for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
            if label_index != -100:
                pred_tag.append(index_to_tag[pred_index])
            pred_list.append(pred_tag)
    for example, pred in zip(examples, pred_list):
        one_sample_result = []
        for one_word, label_token in zip(example, pred):
            one_sample_result.append((one_word, label_token))
        result_list.append(one_sample_result)
    return result_list

In [32]:
#sent1 = ' 오 리 온 스 는 리 그 최 정 상 급 포 인 트 가 드 김 동 훈 을 앞 세 우 는 빠 른 공 수 전 환 이 돋보 이 는 팀 이 다 '
#sent2 = ' 하 이 신 사 에 속 한 섬 들 도 위 로 솟 아 있 는 데 타 인 은 살 고 있 어 요 '
sent1 = '설악산, 오색리구간'
#sent1 = '10시 10분'
test_samples = [sent1]
result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)
result_list

100%|██████████| 1/1 [00:00<00:00, 2336.66it/s]


1/1 [==============================] - 1s 833ms/step


[[('설악산,', 'B_LC'), ('오색리구간', 'B_LC')]]